In [1]:
import pandas as pd
data = pd.read_csv("abstracts.csv")

In [2]:
data = data.drop('Unnamed: 0', axis=1)
data

,S,P,O,label,dboSource,Abstracts
0,http://yago-knowledge.org/resource/Van_Helsing...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Van Helsing (film),Van_Helsing_(film),Van Helsing is a 2004 action gothic horror fil...
1,http://yago-knowledge.org/resource/One_Flew_Ov...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,One Flew Over the Cuckoo's Nest (film),One_Flew_Over_the_Cuckoo's_Nest_(film),One Flew Over the Cuckoo's Nest is a 1975 Amer...
2,http://yago-knowledge.org/resource/The_WB,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,The WB,The_WB,"The WB Television Network (for Warner Bros., o..."
3,http://yago-knowledge.org/resource/This_Must_B...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,This Must Be the Place (film),This_Must_Be_the_Place_(film),This Must Be the Place is a 2011 Italian-Frenc...
4,"http://yago-knowledge.org/resource/Monsters,_Inc.",http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Monsters, Inc.","Monsters,_Inc.","Monsters, Inc. (also known as Monsters, Incorp..."
...,...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Lincoln College, Oxford","Lincoln_College,_Oxford","Lincoln College (formally, The College of the ..."
12759,http://yago-knowledge.org/resource/104th_Unite...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,104th United States Congress,104th_United_States_Congress,The 104th United States Congress was a meeting...
12760,http://yago-knowledge.org/resource/London_Busi...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,London Business School,London_Business_School,London Business School (LBS) is a business sch...
12761,http://yago-knowledge.org/resource/Political_s...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Political science,Political_science,Political science is the scientific study of p...


In [7]:
import spacy    
pos_explanation = spacy.explain('AUX')
print(pos_explanation)

## Pretrained Model Attempt 

In [22]:
from openie import StanfordOpenIE

# https://stanfordnlp.github.io/CoreNLP/openie.html#api
# Default value of openie.affinity_probability_cap was 1/3.
properties = {
    'openie.affinity_probability_cap': 1 / 3,
}
triples = []
with StanfordOpenIE(properties=properties) as client:
    text = data.Abstracts[444]
    print('Text: %s.' % text)
    for triple in client.annotate(text):
        print('|-', triple)
        triples.append(triple)

Text: Pearl Harbor is a 2001 American romantic war drama film directed by Michael Bay, produced by Bay and Jerry Bruckheimer and written by Randall Wallace. It stars Ben Affleck, Kate Beckinsale, Josh Hartnett, Cuba Gooding Jr., Tom Sizemore, Jon Voight, Colm Feore, and Alec Baldwin. The film features a heavily fictionalized version of the attack on Pearl Harbor by Japanese forces on December 7, 1941, focusing on a love story set amidst the lead up to the attack, its aftermath, and the Doolittle Raid. The film was a box office success, earning $59 million in its opening weekend and nearly $450 million worldwide, but received generally negative reviews from critics, who criticized the story, long runtime, screenplay and dialogue, pacing, performances and historical inaccuracies, although the visual effects and Hans Zimmer's score were praised. It was nominated for four Academy Awards, winning in the category of Best Sound Editing. However, it was also nominated for six Golden Raspberry 

In [27]:
#doc = nlp(text)
#for sents in doc.sents:
    #for sent in sents.ents:
        #print(sent)
data.label[444]     

'Pearl Harbor (film)'

In [28]:
from transformers import pipeline
import spacy
unmasker = pipeline('fill-mask', model='bert-base-uncased')
name = triples[0]['subject']
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []


for triple in triples:
    # turn triple into string 
    string = (triple['subject'],triple['relation'],triple['object'])
    string = (' '.join(string))
    nlp = spacy.load("en_core_web_sm")
    tokens = nlp(string)
    
    # if something in chunk
    if len(chunks) > 0:
        
        # if subject related to entity
        if any(good['subject'].casefold() == item for item in item_list) or good['subject'] in name:
            # if chunk is 'is' or 'was'
            if chunks[-1] == 'is' or chunks[-1] == 'was':
                ob = nlp(good['object'])
               
                # get noun chunks from object
                for b in ob.ents:
                    slm = ("%s %s [MASK] %s" %(data.label[444], chunks[-1], b.text))
                    if ('%s - %s - %s' %(name, unmasker(slm)[0]['token_str'], b.text)) not in trips:
                        trips.append('%s - %s - %s' %(name, unmasker(slm)[0]['token_str'], b.text))
                        
                for b in ob.noun_chunks:
                    # make triple
                    slm = ("%s %s [MASK] %s" %(data.label[444], chunks[-1], b.text))
                    # check not already in triples
                    if ('%s - %s - %s' %(name, unmasker(slm)[0]['token_str'], b.text)) not in trips:
                        trips.append('%s - %s - %s' %(name, unmasker(slm)[0]['token_str'], b.text))

            else:
                ob = nlp(good['object'])
                
                for b in ob.ents:
                    # mkae triple
                    slm = ("%s [MASK] %s %s" %(data.label[444], chunks[-1], b.text))
                    # if MASK is punctuation, remove MASK
                    if unmasker(slm)[0]['token_str'] in punc:
                        if ('%s - %s - %s' %(name, chunks[-1], b.text)) not in trips:
                            # check not already in triples
                            trips.append('%s - %s - %s' %(name, chunks[-1], b.text))

                    else:
                        if ('%s - %s%s - %s' %(name, unmasker(slm)[0]['token_str'], chunks[-1].capitalize(), b.text)) not in trips:
                            # check not already in triples
                            trips.append('%s - %s%s - %s' %(name, unmasker(slm)[0]['token_str'], chunks[-1].capitalize(), b.text))
                
                # get noun chunks from object
                for b in ob.noun_chunks:
                    # mkae triple
                    slm = ("%s [MASK] %s %s" %(data.label[444], chunks[-1], b.text))
                    # if MASK is punctuation, remove MASK
                    if unmasker(slm)[0]['token_str'] in punc:
                        if ('%s - %s - %s' %(name, chunks[-1], b.text)) not in trips:
                            # check not already in triples
                            trips.append('%s - %s - %s' %(name, chunks[-1], b.text))

                    else:
                        if ('%s - %s%s - %s' %(name, unmasker(slm)[0]['token_str'], chunks[-1].capitalize(), b.text)) not in trips:
                            # check not already in triples
                            trips.append('%s - %s%s - %s' %(name, unmasker(slm)[0]['token_str'], chunks[-1].capitalize(), b.text))
        
        # empty chunk list
        chunks = []

    # get chunk head 
    for chunk in tokens.noun_chunks:
        chunks.append(chunk.root.head.text)
       
    #print(triple)
    good = triple
    
for trip in trips:
    print(trip)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Pearl Harbor - directed - 2001
Pearl Harbor - directed - American
Pearl Harbor - directed - 2001 American romantic war drama film
Pearl Harbor - producedBy - 2001
Pearl Harbor - by - American
Pearl Harbor - directedBy - Michael Bay
Pearl Harbor - inspiredBy - 2001 American war drama film
Pearl Harbor - a - 2001
Pearl Harbor - a - 2001 romantic war drama film
Pearl Harbor - inspiredBy - 2001 war drama film
Pearl Harbor - inspiredBy - 2001 American romantic war drama film
Pearl Harbor - directed - 2001 war drama film
Pearl Harbor - an - American
Pearl Harbor - a - 2001 American romantic war drama film
Pearl Harbor - inspiredBy - 2001 romantic war drama film
Pearl Harbor - directed - 2001 American war drama film
Pearl Harbor - a - 2001 American war drama film
Pearl Harbor - a - 2001 war drama film
Pearl Harbor - bayBy - Bay
Pearl Harbor - directed - 2001 romantic war drama film
Pearl Harbor - stars - Ben Affleck
Pearl Harbor - for - four
Pearl Harbor - nominatedFor - Academy Awards
Pearl 

## Example output 

In [79]:
data.Abstracts[0]

"Van Helsing is a 2004 action gothic horror film written and directed by Stephen Sommers. It stars Hugh Jackman as Dutch monster hunter Van Helsing, and Kate Beckinsale as Anna Valerious. Van Helsing is both a homage and tribute to the Universal Horror Monster films from the 1930s and 1940s (also produced by Universal Pictures which were in turn partially based on novels by Bram Stoker and Mary Shelley), of which Sommers is a fan. The eponymous character was inspired by the Dutch vampire hunter Abraham Van Helsing from Irish author Bram Stoker's novel Dracula. Distributed by Universal Pictures, the film includes a number of monsters such as Count Dracula (and other vampires), Frankenstein's monster, Duergar, Mr. Hyde and werewolves in a way similar to the multi-monster movies that Universal produced in the 1940s, such as Frankenstein Meets the Wolf Man, House of Frankenstein and House of Dracula. The film received mostly negative reviews but grossed $300 million worldwide against a bud

Van Helsing (film) - a - 2004 action gothic horror film
Van Helsing (film) - inspiredby - 2004 action horror film
Van Helsing (film) - directedby - Stephen Sommers
Van Helsing (film) - a - gothic
Van Helsing (film) - a - 2004 action horror film
Van Helsing (film) - inspiredby - 2004 action gothic horror film
Van Helsing (film) - as - monster hunter Van Helsing
Van Helsing (film) - as - Dutch monster hunter Van Helsing
Van Helsing (film) - hughJackman - Hugh Jackman
Van Helsing (film) - accordingto - Universal Horror Monster films
Van Helsing (film) - inspiredby - homage
Van Helsing (film) - producedby - Universal Pictures
Van Helsing (film) - produced - homage
Van Helsing (film) - produced - Universal Horror Monster films
Van Helsing (film) - producedby - Universal Horror Monster films
Van Helsing (film) - builtby - 1930s
Van Helsing (film) - an - homage
Van Helsing (film) - from - homage
Van Helsing (film) - adaptedfrom - Universal Horror Monster films
Van Helsing (film) - from - 1930s
Van Helsing (film) - to - homage

## Testing 

Frost/Nixon (film) - a - 2008 historical drama film
Frost/Nixon (film) - a - 2008 drama film
Frost/Nixon (film) - thebehind - Frost Nixon interviews
Frost/Nixon (film) - tells - story
Frost/Nixon (film) - thebehind - Frost Nixon interviews
Frost/Nixon (film) - directedby - Ron Howard
Frost/Nixon (film) - producedby - Universal Pictures
Frost/Nixon (film) - directedby - Howard
Frost/Nixon (film) - directedby - Howard
Frost/Nixon (film) - reunites - its original two stars
Frost/Nixon (film) - reunites - its two stars
Frost/Nixon (film) - filmedin - United States
Frost/Nixon (film) - basedon - United States
Frost/Nixon (film) - releasedon - December
Frost/Nixon (film) - releasedon - December
Frost/Nixon (film) - failedat - box office
Frost/Nixon (film) - Despite - critical acclaim

In [2]:
#

In [ ]:
import spacy 

In [3]:
                                                                                                                                              
import claucy     

In [15]:
                                                                                                                                          
nlp = spacy.load("en_core_web_sm")
claucy.add_to_pipe(nlp)                                                                                                                                     
doc = nlp("""London is the capital and largest city of England and the United Kingdom, with a population of just under 9 million. It stands on the River Thames in south-east England at the head of a 50-mile (80 km) estuary down to the North Sea, and has been a major settlement for two millennia. The City of London, its ancient core and financial centre, was founded by the Romans as Londinium and retains its medieval boundaries. The City of Westminster, to the west of the City of London, has for centuries hosted the national government and parliament. Since the 19th century, the name "London" has also referred to the metropolis around this core, historically split between the counties of Middlesex, Essex, Surrey, Kent, and Hertfordshire, which largely comprises Greater London, governed by the Greater London Authority. As one of the world's major global cities, London exerts a strong influence on its arts, entertainment, fashion, commerce and finance, education, health care, media, science and technology, tourism, and transport and communications. Its GDP (€801.66 billion in 2017) makes it the largest urban economy in Europe, and it is one of the major financial centres in the world. With Europe's largest concentration of higher education institutions, it is home to some of the highest-ranked academic institutions in the world—Imperial College London in natural and applied sciences, the London School of Economics in social sciences, and the comprehensive University College London. London is the most visited city in Europe and has the busiest city airport system in the world. The London Underground is the oldest rapid transit system in the world. London is home to the most 5-star hotels of any city. London's diverse cultures encompass over 300 languages. The mid-2018 population of Greater London of about 9 million made it Europe's third-most populous city, accounting for 13.4% of the population of the United Kingdom and over 16% of the population of England. The Greater London Built-up Area is the fourth-most populous in Europe with about 9.8 million inhabitants at the 2011 census. The London metropolitan area is the third-most populous in Europe with about 14 million inhabitants in 2016, granting London the status of a megacity. London has four World Heritage Sites: the Tower of London; Kew Gardens; the combined Palace of Westminster, Westminster Abbey, and St Margaret's Church; and also the historic settlement in Greenwich, where the Royal Observatory, Greenwich defines the prime meridian (0° longitude) and Greenwich Mean Time. Other landmarks include Buckingham Palace, the London Eye, Piccadilly Circus, St Paul's Cathedral, Tower Bridge, and Trafalgar Square. London has many museums, galleries, libraries and cultural venues, including the British Museum, National Gallery, Natural History Museum, Tate Modern, British Library, and numerous West End theatres. Important sporting events held in London include the FA Cup Final (held annually at Wembley Stadium), Wimbledon Tennis Championships and the London Marathon. In 2012, London became the first city to host three Summer Olympic Games.""")
for sent in doc.sents:
    propositions = sent._.clauses[0].to_propositions(as_text=False)                                                                                               
    print(propositions)   

[(London, is, largest city), (London, is, the capital, largest city), (London, is, the capital)]
[(It, stands, estuary)]
[(The City, 'is', its ancient core and financial centre)]
[(The City, has)]
[(the name "London, referred, also), (the name "London, referred, Since the 19th century), (the name "London, referred, to the metropolis around this core), (the name "London, referred, Since the 19th century, also, to the metropolis around this core)]
[(London, exerts, a strong influence, As one of the world's major global cities), (London, exerts, a strong influence)]
[(Its GDP, 'is', billion)]
[(it, is, home)]
[(London, is, the most visited city)]
[(The London Underground, is, the oldest rapid transit system)]
[(London, is, home)]
[(London's diverse cultures, encompass, over 300 languages)]
[(The mid-2018 population, made, Europe's third-most populous city)]
[(The Greater London Built-up Area, is, the fourth-most populous)]
[(The London metropolitan area, is, the third-most populous)]
[(Lo

In [1]:
# take head.text 
# unmasker (___ head [MASK] END)
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("London [MASK] Largest city")

/Users/ab/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.5794567465782166,
  'token': 1011,
  'token_str': '-',
  'sequence': 'london - largest city'},
 {'score': 0.17109297215938568,
  'token': 1024,
  'token_str': ':',
  'sequence': 'london : largest city'},
 {'score': 0.10919811576604843,
  'token': 1010,
  'token_str': ',',
  'sequence': 'london, largest city'},
 {'score': 0.09484130889177322,
  'token': 1996,
  'token_str': 'the',
  'sequence': 'london the largest city'},
 {'score': 0.004617173690348864,
  'token': 2003,
  'token_str': 'is',
  'sequence': 'london is largest city'}]

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Van Helsing film was [MASK] in 2004")

In [ ]:
import spacy
text1 = """Dwayne Douglas Johnson (born May 2, 1972), also known by his ring name The Rock, 
is an American actor and former professional wrestler. Widely regarded as one of the greatest professional wrestlers of all time, 
he was integral to the development and success of the WWE during the Attitude Era, an industry boom period in the late 1990s and early 2000s. 
Johnson wrestled for WWF/E for eight years prior to pursuing an acting career. 
His films have grossed over $3.5 billion in North America and over $10.5 billion worldwide, 
making him one of the world's highest-grossing and highest-paid actors. 
Prior to his emergence as a top-grossing actor beginning in the 2010s, Johnson was an athlete. 
At Freedom High School in Bethlehem Township, Pennsylvania in the Lehigh Valley, 
he ultimately emerged as a standout on the school's football and wrestling teams, which competed in the Eastern Pennsylvania Conference, 
a conference known for producing top professional and Olympic athletes. 
He credits his high school football coach with helping inspire transformational changes in his life, 
which before entering sports in 11th grade had been characterized by fights and petty crimes as a high-school teenager. 
His football capabilities at Freedom High School led him to be ranked as one of the top ten defensive tackle high-school prospects in the nation, 
drawing the attention of several NCAA Division I college football programs. 
Johnson accepted an athletic scholarship to play at the University of Miami, 
one of the country's top college football programs. As a college freshman, 
he was a member of the University of Miami's 1991 national championship team but largely spent his four college football years at the University of Miami in backup roles behind elite players, 
including future NFL star and Pro Football Hall of Fame inductee Warren Sapp. 
Johnson aspired to play professional football but went undrafted in the 1995 NFL Draft. 
Following his graduation from the University of Miami that year, 
he signed with the Calgary Stampeders of the Canadian Football League but was cut from the team in his first season. 
In 1996, Johnson's father Rocky, a longtime professional wrestler, 
assisted in helping him secure a contract with the World Wrestling Federation (WWF, now WWE). Johnson quickly rose to global prominence, 
aided by a gimmick he employed as a charismatic trash-talker. The role ushered in the Attitude Era in professional wrestling, 
which contributed to the industry's boom period in the late 1990s and early 2000s. 
Johnson left WWE in 2004 and returned in 2011 as a part-time performer until 2013, 
making sporadic appearances until retiring in 2019. 
A 10-time world champion, including the promotion's first of African-American descent, 
he is also a two-time Intercontinental Champion, a five-time Tag Team Champion, the 2000 Royal Rumble winner, 
and WWE's sixth Triple Crown champion. 
Johnson headlined the most-bought professional wrestling pay-per-view (WrestleMania XXVIII) and was featured among the most watched episodes of WWE's flagship television series (Raw and SmackDown). 
Johnson's first film appearance was in The Mummy Returns (2001). 
The following year he played his first leading role as the titular character in the sword and sorcery film The Scorpion King (2002). 
He has since starred in family films The Game Plan (2007), Race to Witch Mountain (2009), Tooth Fairy (2010), and Jungle Cruise (2021), 
as well as the action adventure films Journey 2: The Mysterious Island (2012), G.I. Joe: Retaliation (2013), Hercules (2014), Skyscraper (2018), 
San Andreas (2015) and Rampage (2018). He also starred in the action comedy films Get Smart (2008), Central Intelligence (2016), Baywatch (2017), 
and Red Notice (2021). In 2016 he voiced Maui in the Disney animated film Moana (2016). His role as Luke Hobbs in the Fast & Furious films, 
beginning with Fast Five (2011), helped the franchise become one of the highest-grossing in film. 
He subsequently starred in the spin-off Hobbs & Shaw (2019). Johnson also stars in the Jumanji films, 
appearing in Jumanji: Welcome to the Jungle (2017) and Jumanji: The Next Level (2019), and voices Bark Kent / Krypto / Superdog, Anubis, and Teth-Adam / Black Adam in the animated film DC League of Super-Pets (2022), 
before starring as the latter in the live-action superhero film Black Adam (2022). 
Johnson produced and starred in the HBO comedy-drama series Ballers (2015–2019) and stars and produces the autobiographical sitcom Young Rock (2021). 
In 2000, Johnson released his autobiography The Rock Says, which was a New York Times bestseller. 
In 2012, he co-founded the entertainment production company Seven Bucks Productions and is co-owner of the XFL, a professional American football league. 
In 2016 and 2019, Johnson was named by Time one of the world's most influential people. """
sent_split = sent_tokenize(text1)
for sent in sent_split:
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sent)
    for token in doc:
        print('text:',token.text,'- dep:', token.dep_,'- head:', token.head.text,'- pos:', token.head.pos_, '- children:',
                [child for child in token.children])
    print()

In [ ]:
for sent in sent_split:
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sent)
    for chunk in doc.noun_chunks:
        print('text:', chunk.text, '- root:', chunk.root.text,'- dep:', chunk.root.dep_, '- root head:',
                chunk.root.head.text)
    print()